In [25]:
%load_ext autoreload
%autoreload 2
import os
from gigachat_model import GigaChatLM
# from gigachat_model import gigachat_completion
from langchain.prompts.chat import (
    AIMessagePromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.schema import AIMessage, HumanMessage, SystemMessage

gc = GigaChatLM()

# creds = '2f2e446b-9532-4de4-9df4-5632872dd38f'
api_key = "ZTZkM2ZmODYtNDRmNC00OWQ0LTkyNTUtOTA1NzE1ZGY2ZTFjOjJmMmU0NDZiLTk1MzItNGRlNC05ZGY0LTU2MzI4NzJkZDM4Zg=="
# os.environ[''] = creds
os.environ['GIGACHAT_API_KEY'] = api_key


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
os.environ.get("GIGACHAT_API_KEY")

'ZTZkM2ZmODYtNDRmNC00OWQ0LTkyNTUtOTA1NzE1ZGY2ZTFjOjJmMmU0NDZiLTk1MzItNGRlNC05ZGY0LTU2MzI4NzJkZDM4Zg=='

In [22]:
messages = [
    SystemMessage(
        content="Ты полезный ассистент, который умеет переводить русский на английский."
    ),
    HumanMessage(content="Переведи это предложение. Я люблю программирование."),
]


In [65]:
import re

async def gigachat_completion(
    client, #: gigachat.GigaChat,
    model: str,
    prompt: str,
    max_tokens: int,
    temperature: float,
    system: str,
    stopwords: list[str],
    **kwargs,
) -> str:
    try:
        import gigachat
    except ModuleNotFoundError:
        raise Exception(
            "attempted to use 'gigachat' LM type, but package `gigachat` is not installed. \
please install gigachat via `pip install 'gigachat'`",
        )
    messages=[]
    if system:
        messages.append(
            gigachat.models.Messages(
                role=gigachat.models.MessagesRole.SYSTEM,
                content=system,
            )  
        )
        
    messages.append(
            gigachat.models.Messages(
                role=gigachat.models.MessagesRole.USER,
                content=prompt,
            )
            )
    print(f'Messages: {messages}')
    async def completion():
        
        payload = gigachat.models.Chat(
            messages=messages,
            model=model,
            max_tokens_to_sample=max_tokens,
            # max_tokens=max_tokens,
            temperature=temperature,
            **kwargs,
            update_display=0.1
        )

        # response = client.chat(
        #     payload
        # )

        total_text = ''
        pattern = '|'.join(stopwords)
        # start = time.time()
        async with GigaChat(credentials=os.environ.get("GIGACHAT_API_KEY"), scope="GIGACHAT_API_CORP", verify_ssl_certs=False) as giga:
        # stop = ['']
            async for chunk in giga.astream(payload):
                text = chunk.choices[0].delta.content
                total_text += text
                print(f'Total text: {total_text}')
                k=re.search(pattern, total_text)
                if k:
                    break


        if k:
            goodText = total_text[:k.span()[1]]
            print(goodText)

        return total_text
    answer = await completion()
    return answer

await gigachat_completion(gc.client,gc.model,'привет. Как твои дела? Какая сейчас погода в Люксембурге. QQQ',256,0.8,'переведи на английский. В конце добавь "QQQ"')

Messages: [Messages(role='system', content='переведи на английский. В конце добавь "QQQ"', function_call=None, name=None, attachments=None, id_=None), Messages(role='user', content='привет. Как твои дела? Какая сейчас погода в Люксембурге. QQQ', function_call=None, name=None, attachments=None, id_=None)]


2024-06-03:12:40:03,611 INFO     [_client.py:1026] HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


'Hello. How are you doing? What is the weather like in Luxembourg now? QQQ'

In [50]:
import asyncio
import time
import gigachat
from gigachat import GigaChat
from gigachat.models import Chat, Messages, MessagesRole

messages=[]
system = "Ты самый лучший в мире писатель детских сказок "
prompt = "Как звали чипполино из одноименной книги:?"
if system:
    messages.append(
        gigachat.models.Messages(
            role=gigachat.models.MessagesRole.SYSTEM,
            content=system,
        )  
    )
    
messages.append(
        gigachat.models.Messages(
            role=gigachat.models.MessagesRole.USER,
            content=prompt,
        )
        )
print(f'Messages: {messages}')

payload = gigachat.models.Chat(
            messages=messages,
            model=gc.model,
            max_tokens_to_sample=512,
            # max_tokens=max_tokens,
            temperature=0.8,
            update_display=0.1
        )

async def main():
    async with GigaChat(credentials=os.environ.get("GIGACHAT_API_KEY"),
            scope="GIGACHAT_API_CORP",
            verify_ssl_certs=False) as giga:
        async for chunk in giga.astream(payload):
            print(time.time(), chunk, flush=True)

asyncio.run(main())



Messages: [Messages(role='system', content='Ты самый лучший в мире писатель детских сказок ', function_call=None, name=None, attachments=None, id_=None), Messages(role='user', content='Как звали чипполино из одноименной книги:?', function_call=None, name=None, attachments=None, id_=None)]


RuntimeError: asyncio.run() cannot be called from a running event loop

In [38]:
import time
max_tokens=256
temperature=0.8
system = "Ты полезный ассистент, который умеет переводить русский на английский. "
    
start = time.time()
response = gigachat_completion(
    client=gc.client,
    model=gc.model,
    prompt='Привет',
    # max_tokens_to_sample=max_gen_toks,
    # max_tokens=self.max_tokens,
    temperature=temperature,  
    system=system,
    max_tokens=max_tokens,
    # **self.kwargs,
)

response
end = time.time()
print(end-start)

Messages: [Messages(role='system', content='Ты полезный ассистент, который умеет переводить русский на английский. ', function_call=None, name=None, attachments=None, id_=None), Messages(role='user', content='Привет', function_call=None, name=None, attachments=None, id_=None)]
0.0002639293670654297


In [ ]:
def completion():
    
    payload = gigachat.models.Chat(
        messages=messages,
        model=model,
        max_tokens_to_sample=max_tokens,
        # max_tokens=max_tokens,
        temperature=temperature,
    )
    response = client.chat(
        payload
    )
    return response.choices[0].message.content